# Template matching

Iz originalnih slika je za svaku klasu izrezan po jedan logo, a potom su korištenjem metode Template matchinga, izolirani ground Truth za svaku sliku i kropani prema  imenu slike. Ručno su pregledani cropani logoi i oni koji nisu odgovarali površinom i izgledom su izbrisani. Proces je trajao par sati za ukupno 45k slika. Nakon toga smo dobili cropane logoe koji su se zvali identično kao i slike iz kojih su cropani. Kasnije ćemo te cropane logoe i njihove lokacije koristiti za 

In [ ]:
import cv2
import sys
import os
import numpy as np
from skimage.measure import compare_ssim as ssim
import datetime

#THRESH = (400*216)/75
IMGDIR = '/home/bosnmar/Programming/source/'
CROPLOGO = '/home/bosnmar/Programming/croped/'
newHeight = 400
newWidth = 220
THRESH = (newHeight*newWidth)/50
picture = 0
METHOD='cv2.TM_CCOEFF_NORMED'
SAVELOCATION = '/home/bosnmar/Programming/Logos/'

def preprocessPicture(picture_path):
    loadImage = cv2.imread(picture_path,0)

    #gray_image = cv2.cvtColor(loadImage, cv2.COLOR_BGR2GRAY)
   
    #loadImage  = cv2.resize(loadImage, (newWidth, newHeight))
    #image = image.flatten()
    #x = int(newWidth/10)
    #w = int((8*newWidth)/10)
    image = loadImage[0:round((loadImage.shape[1]/2)), 20:loadImage.shape[0]-20]
    median_image = cv2.medianBlur(image,3)

    return median_image

def preprocessLogo(logo_path):
    loadLogo = cv2.imread(logo_path,0)
    logo = cv2.equalizeHist(loadLogo)
 
    median_logo = cv2.medianBlur(logo, 3)
    #ret, binary_logo = cv2.threshold(median_logo, 127,255, cv2.THRESH_BINARY)

    return median_logo

def searchImage(image,  template, imgName, direct):

    
    w, h = template.shape[::-1]
    method =eval(METHOD)

    try:
        res = cv2.matchTemplate(image,template,method)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

        top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)

    #cv2.rectangle(image,top_left, bottom_right, 255, 2)
    #cv2.rectangle(imOut, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)

        crop_img = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]

        location = SAVELOCATION + direct + '/' + str(imgName)

        cv2.imwrite(location, crop_img)
    except:
        print(imgName)
    """
    cv2.imshow("Output Image", crop_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()"""





if __name__ == "__main__":
    
    #make cv use all the process powert available
    cv2.setUseOptimized(True)
    cv2.setNumThreads(4)
    template = preprocessLogo('/home/bosnmar/Programming/albertsons.jpg')
    """
    image = preprocessPicture('/home/bosnmar/Programming/test_image.jpg')
    template = preprocessLogo('/home/bosnmar/Programming/albertsons.jpg')
    w, h = template.shape[::-1]

    test_img = image.copy()
    method = eval(METHOD)

    res = cv2.matchTemplate(test_img,template,method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)


    cv2.rectangle(test_img,top_left, bottom_right, 255, 2)

    cv2.imshow("Output", test_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    images = [image for image in os.listdir(IMGDIR)]
    start = datetime.datetime.now()
    print ("Ukupno za obrtaditi {} slika".format(len(images)))

    for image in images:
        picture = picture +1 
        search_image = preprocessPicture(IMGDIR + image)
        searchImage(search_image, template, picture)
        if picture % 50 == 0:
            print ("Trenutno na: {}".format(picture))
        
        #if picture > 50:
            #break

    end = datetime.datetime.now()

    print ("Started at: {}".format(start))
    print ("Ended at: {}".format(end))"""


    baseClasses = [direct for direct in os.listdir(IMGDIR)]
    print (baseClasses)

    for direct in baseClasses:
        location = SAVELOCATION + direct + '/'
        if os.path.exists(location) is not True:
            os.makedirs(location)
    
    logos = [logo for logo in os.listdir(SAVELOCATION)]

    cropedLogos = [logo for logo in os.listdir(CROPLOGO)]
   
    i = 0
    """
    for direct in baseClasses:
        if direct in cropedLogos and direct in logos:
            print ("Source: {} exists in all thre arrays".format(direct))
            i = i+1

            print(CROPLOGO + direct + '.jpeg')
        
    if i == len(baseClasses):
        print("Alles clar")

    print (baseClasses)
    print (logos)
    print (cropedLogos)
    
    """

    start = datetime.datetime.now()

    for direct in baseClasses:
        picture = 0
        logoImg = preprocessLogo(CROPLOGO + direct)
        print ("Working on {}".format(direct))
        for image in os.listdir(IMGDIR+direct + '/'):
            picture = picture + 1
            search_image = preprocessPicture(IMGDIR + direct + '/'+ image)
            searchImage(search_image, logoImg, image, direct)
            if picture % 50 == 0:
                print ("Trenutno na: {}".format(picture))

    end = datetime.datetime.now()

    print ("Started at: {}".format(start))
    print ("Ended at: {}".format(end)) 

